In [3]:
import csv
import time  # Just to compare fit times
from pathlib import Path
from pprint import pprint

In [2]:
import numpy as np
import ray
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tune_sklearn import TuneSearchCV
from xgboost import XGBRegressor
import pandas as pd

In [12]:
%load_ext memory_profiler

In [13]:
!/home/dev/Desktop/Work/uncoverml/venv/bin/ray start --head --port=6379 --num-cpus=1

Local node IP: 192.168.11.161
2021-02-03 00:59:53,550	INFO services.py:1171 -- View the Ray dashboard at http://localhost:8265

--------------------
Ray runtime started.
--------------------

Next steps
  To connect to this Ray runtime from another node, run
    ray start --address='192.168.11.161:6379' --redis-password='5241590000000000'
  
  Alternatively, use the following Python code:
    import ray
    ray.init(address='auto', _redis_password='5241590000000000')
  
  If connection fails, check your firewall settings and network configuration.
  
  To terminate the Ray runtime, run
    ray stop


In [14]:
!/home/dev/Desktop/Work/uncoverml/venv/bin/ray --version

ray, version 1.1.0


In [15]:
ray.__version__

'1.1.0'

In [8]:
!/home/dev/Desktop/Work/uncoverml/venv/bin/ray stop

Did not find any active Ray processes.


In [ ]:
ray.init(address='192.168.11.161:6379', _redis_password='5241590000000000')

2021-02-03 01:00:06,837	INFO worker.py:656 -- Connecting to existing Ray cluster at address: 192.168.11.161:6379


In [13]:
ray.shutdown()

In [10]:
# read the csv file created

from pathlib import Path
ray.init(num_cpus=24, num_gpus=4)

input_files = [
    Path("../data/formated_dataset.csv"),
    Path("../data/scaler_df.csv"),
    Path("../data/quantile_df.csv"),
    Path("../data/max_abs_df.csv")
]


input_file = input_files[2]
print("Input file: ",input_file)
df = pd.read_csv(input_file).astype('float32')
df = df[~df.isin([np.nan, np.inf, -np.inf,-9999.0]).any(1)]
y = df['target']
X = df.drop("target",axis=1)

# Set training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

2021-02-03 01:05:11,382	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


Input file:  ../data/quantile_df.csv


In [14]:
import time
parameters = {
    "objective": ["reg:gamma", "reg:squarederror"],
    "gamma":[0,5],
    "eval_metric": ['logloss', "rmse"],
    "n_estimators":[300, 500],
    "max_depth":[3, 15],
    "subsample":[0.5, 1],
    "min_child_weight":[20, 80],
    "learning_rate":[0.01, 0.2],
    "colsample_bytree":[0.5,1]
}

# XGBRegressor(colsample_bylevel=1)
# model=XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', gpu_id=0, n_jobs=-1)
model=XGBRegressor(colsample_bylevel=1, tree_method='gpu_hist', gpu_id=0, n_jobs=-1)
xgb_tune_search = TuneSearchCV(
    model,
    parameters,

    search_optimization="bayesian",
    n_jobs=-1,
    n_trials=200,
    early_stopping=True,

    verbose=2,
    return_train_score=True,
    loggers=["csv"],
    use_gpu=True
)

start = time.time()
xgb_tune_search.fit(X_train, y_train)
end = time.time()
print("Tune Fit Time:", end - start)
y_pred = xgb_tune_search.predict(X_test)
print("Predicted value: ", y_pred)

#OOS sample dataset

/home/dev/Desktop/Work/uncoverml/venv/lib/python3.8/site-packages/tune_sklearn/tune_basesearch.py:426: UserWarning: early_stopping is enabled but max_iters = 1. To enable partial training, set max_iters > 1.
  warnings.warn(
/home/dev/Desktop/Work/uncoverml/venv/lib/python3.8/site-packages/tune_sklearn/tune_basesearch.py:431: UserWarning: tune-sklearn implements incremental learning for xgboost models following this: https://github.com/dmlc/xgboost/issues/1686. This may negatively impact performance. To disable, set `early_stopping=False`.
  warnings.warn(
2021-02-03 01:09:25,851	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: cannot pickle 'dict_values' object


2021-02-03 01:09:38,222	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: cannot pickle 'dict_values' object


Trial _Trainable_41ad1257 reported split0_test_score=0.6906900313670497,split1_test_score=0.689567782167332,split2_test_score=0.6865544277345781,split3_test_score=0.6914061098638885,split4_test_score=0.6890001381545954,average_test_score=0.6894436978574887,split0_train_score=0.6975074177253979,split1_train_score=0.6958666594310917,split2_train_score=0.693828584215743,split3_train_score=0.6961241148641593,split4_train_score=0.6965616028204296,average_train_score=0.6959776758113644,objective=0.6894436978574887 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.XGB: 4>, 'X_id': ObjectRef(ffffffffffffffffffffffff0100000005000000), 'y_id': ObjectRef(ffffffffffffffffffffffff0100000006000000), 'groups': None, 'cv': KFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': <function _passthrough_scorer at 0x7f2c0198ee50>}, 'max_iters': 1, 'return_train_score': True, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_list': [XG

Trial _Trainable_41ad1256 reported split0_test_score=0.71981926817712,split1_test_score=0.7163325858212375,split2_test_score=0.716880922943848,split3_test_score=0.7187586804145882,split4_test_score=0.7149791223338996,average_test_score=0.7173541159381386,split0_train_score=0.7285876268552156,split1_train_score=0.728636551183315,split2_train_score=0.728121114439552,split3_train_score=0.7268555221432443,split4_train_score=0.727375976151289,average_train_score=0.7279153581545231,objective=0.7173541159381386 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.XGB: 4>, 'X_id': ObjectRef(ffffffffffffffffffffffff0100000005000000), 'y_id': ObjectRef(ffffffffffffffffffffffff0100000006000000), 'groups': None, 'cv': KFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': <function _passthrough_scorer at 0x7f2c0198ee50>}, 'max_iters': 1, 'return_train_score': True, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_list': [XGBReg

2021-02-03 01:09:48,907	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: cannot pickle 'dict_values' object


...

Trial _Trainable_e603a0ea reported split0_test_score=0.8985883884083683,split1_test_score=0.8999195516524643,split2_test_score=0.8978582566413835,split3_test_score=0.898659357573619,split4_test_score=0.899083699666654,average_test_score=0.8988218507884979,split0_train_score=0.9923352749395092,split1_train_score=0.9921929538122677,split2_train_score=0.9922315267700207,split3_train_score=0.9920548189120235,split4_train_score=0.9923347590386881,average_train_score=0.9922298666945017,objective=0.8988218507884979 with parameters={'early_stopping': True, 'early_stop_type': <EarlyStopping.XGB: 4>, 'X_id': ObjectRef(ffffffffffffffffffffffff0100000005000000), 'y_id': ObjectRef(ffffffffffffffffffffffff0100000006000000), 'groups': None, 'cv': KFold(n_splits=5, random_state=None, shuffle=False), 'fit_params': {}, 'scoring': {'score': <function _passthrough_scorer at 0x7f2c0198ee50>}, 'max_iters': 1, 'return_train_score': True, 'n_jobs': 1, 'metric_name': 'average_test_score', 'estimator_list': [XG

2021-02-03 06:08:03,687	WARNING tune.py:427 -- Trial Runner checkpointing failed: cannot pickle 'dict_values' object


Trial name,status,loc,colsample_bytree,eval_metric,gamma,learning_rate,max_depth,min_child_weight,n_estimators,objective,subsample,iter,total time (s),split0_test_score,split1_test_score,split2_test_score
_Trainable_41ad1256,TERMINATED,,0.993914,logloss,4,0.106098,5,48,427,reg:gamma,0.712492,1,15.301,0.719819,0.716333,0.716881
_Trainable_41ad1257,TERMINATED,,0.810559,rmse,4,0.0618624,8,43,304,reg:squarederror,0.769133,1,11.7017,0.69069,0.689568,0.686554
_Trainable_41ad1258,TERMINATED,,0.642002,rmse,2,0.188684,13,62,353,reg:gamma,0.946834,1,20.3758,0.792647,0.790337,0.79124
_Trainable_41ad1259,TERMINATED,,0.809301,logloss,4,0.0980519,12,56,357,reg:gamma,0.927369,1,22.4201,0.776856,0.776226,0.776699
_Trainable_41ad125a,TERMINATED,,0.920794,rmse,4,0.0594175,13,31,310,reg:squarederror,0.890917,1,14.1671,0.713767,0.714773,0.714012
_Trainable_49180e2e,TERMINATED,,0.777153,logloss,0,0.179051,7,69,450,reg:squarederror,0.772476,1,34.715,0.831292,0.832759,0.829575
_Trainable_4b2b86f0,TERMINATED,,0.592654,logloss,0,0.0685878,6,45,498,reg:squarederror,0.788357,1,22.8396,0.778334,0.777577,0.775818
_Trainable_4e337722,TERMINATED,,0.803954,logloss,5,0.0264657,14,61,479,reg:squarederror,0.941165,1,22.4972,0.705171,0.706328,0.70637
_Trainable_4f7335dc,TERMINATED,,0.764703,rmse,3,0.015557,13,35,344,reg:gamma,0.730438,1,110.676,0.790413,0.790884,0.789557
_Trainable_51ded506,TERMINATED,,0.770986,rmse,4,0.0503386,10,59,410,reg:squarederror,0.865644,1,14.6038,0.704591,0.705436,0.705015


2021-02-03 06:08:03,741	INFO tune.py:448 -- Total run time: 17917.94 seconds (17917.79 seconds for the tuning loop).


Tune Fit Time: 18029.76014471054
Predicted value:  [0.31905305 0.6923849  0.5843311  ... 0.8494585  0.11388404 0.02276994]


In [ ]:
xgb_tune_search.best_estimator_

In [ ]:
#save results
import pickle

fileObj = open('best_estimator_.pkl', 'wb')
pickle.dump(xgb_tune_search.best_estimator_,fileObj)
fileObj.close()

In [ ]:
#save results
import pickle

fileObj = open('xgb_tune_search.pkl', 'wb')
pickle.dump(xgb_tune_search,fileObj)
fileObj.close()

In [ ]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)

    return {'r2': round(r2, 4), 'MAE': round(mean_absolute_error, 4), 'MSE': round(mse, 4), 'RMSE': round(np.sqrt(mse), 4), "explained_variance": round(explained_variance, 4)}

In [ ]:
regression_results(y_test, y_pred)

## Bayesian Hyperparameter Optimization and XGBoost

In [22]:


###############################################
# Import Machine Learning Assets
###############################################
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

###############################################
# Import Miscellaneous Assets
###############################################
import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from functools import partial
from pprint import pprint as pp
from tqdm import tqdm, tqdm_notebook

pd.set_option('display.expand_frame_repr', False)

###############################################
# Declare Global Variables
###############################################
CROSS_VALIDATION_PARAMS = dict(n_splits=5, shuffle=True, random_state=32)
XGBOOST_REGRESSOR_PARAMS = dict(
    learning_rate=0.2, n_estimators=200, subsample=0.8, colsample_bytree=0.8, 
    max_depth=10, n_jobs=-1
)

BAYESIAN_OPTIMIZATION_MAXIMIZE_PARAMS = dict(
    init_points=1,  # init_points=20,
    n_iter=2,  # n_iter=60,
    acq='poi', xi=0.0
)
BAYESIAN_OPTIMIZATION_BOUNDARIES = dict(
    max_depth=(5, 12.99),
    gamma=(0.01, 5),
    min_child_weight=(0, 6),
    scale_pos_weight=(1.2, 5),
    reg_alpha=(4.0, 10.0),
    reg_lambda=(1.0, 10.0),
    max_delta_step=(0, 5),
    subsample=(0.5, 1.0),
    colsample_bytree=(0.3, 1.0),
    learning_rate=(0.0, 1.0)
)
BAYESIAN_OPTIMIZATION_INITIAL_SEARCH_POINTS = dict(
    max_depth=[5, 10],
    gamma=[0.1511, 3.8463],
    min_child_weight=[2.4073, 4.9954],
    scale_pos_weight=[2.2281, 4.0345],
    reg_alpha=[8.0702, 9.0573],
    reg_lambda=[2.0126, 3.5934],
    max_delta_step=[1, 2],
    subsample=[0.8, 0.8234],
    colsample_bytree=[0.8, 0.7903],
    learning_rate=[0.2, 0.1]
)

reserve_features = [
    'rs1_x', 'rs1_y', 'rs2_x', 'rs2_y', 'rv1_x', 'rv1_y', 'rv2_x', 'rv2_y',
    'total_reserve_dt_diff_mean', 'total_reserve_mean', 'total_reserve_sum'
]

BASE_ESTIMATOR = partial(XGBRegressor)
# train_df, test_df = None, None
# oof_predictions, test_predictions = None, None
# train_input = None
# best_round = None
# target = None



ERROR! Session/line number was not unique in database. History logging moved to new session 187


In [23]:
# read the csv file created
input_dataset = Path("../data/quantile_df.csv")
df = pd.read_csv(input_dataset)
df = df.drop(["x","y"],axis=1)


y = df['target'] 
X = df.drop("target",axis=1)

# train_df, test_df = None, NOne
# oof_predictions, test_predictions = None, None
# train_input = None
# best_round = None
# target = None

# X, y = train[:, 1:], train[:, :1]
# y = y.ravel()

# Set training and validation sets
train_df, test_df, target, test_predictions = train_test_split(X, y, test_size=0.33)

In [24]:


def search_node(**kwargs):
    # global train_df, test_df, train_input, oof, test_predictions, best_round, target
    global train_df, target

    ###############################################
    # Unify Parameters
    ###############################################
    received_params = dict(dict(n_estimators=200,), **{_k: _v if _k not in ('max_depth') else int(_v) for _k, _v in kwargs.items()})
    current_params = dict(XGBOOST_REGRESSOR_PARAMS, **received_params)

    ###############################################
    # Initialize Folds and Result Placeholders
    ###############################################
    folds = KFold(**CROSS_VALIDATION_PARAMS)
    evaluation = np.zeros((current_params['n_estimators'], CROSS_VALIDATION_PARAMS['n_splits']))
    oof_predictions = np.empty(len(train_df))
    np.random.seed(32)

    progress_bar = tqdm_notebook(
        enumerate(folds.split(target, target)), 
        total=CROSS_VALIDATION_PARAMS['n_splits'], 
        leave=False
    )
    
    ###############################################
    # Begin Cross-Validation
    ###############################################
    for fold, (train_index, validation_index) in progress_bar:
        train_input, validation_input = train_df.iloc[train_index], train_df.iloc[validation_index]
        train_target, validation_target = target.iloc[train_index], target.iloc[validation_index]

        ###############################################
        # Initialize and Fit Model With Current Parameters
        ###############################################
        model = BASE_ESTIMATOR(**current_params)
        eval_set = [(train_input, train_target), (validation_input, validation_target)]
        model.fit(train_input, train_target, eval_set=eval_set, verbose=False)

        ###############################################
        # Find Best Round for Validation Set
        ###############################################
        evaluation[:, fold] = model.evals_result_["validation_1"]['rmse']
        best_round = np.argsort(evaluation[:, fold])[0]

        progress_bar.set_description('Fold #{}:   {:.5f}'.format(
            fold, evaluation[:, fold][best_round]
        ), refresh=True)

    ###############################################
    # Compute Mean and Standard Deviation of RMSLE
    ###############################################
    mean_eval, std_eval = np.mean(evaluation, axis=1), np.std(evaluation, axis=1)
    best_round = np.argsort(mean_eval)[0]
    search_value = mean_eval[best_round]

    ###############################################
    # Update Best Score and Return Negative Value
    # In order to minimize error, instead of maximizing accuracy
    ###############################################
    print(' Stopped After {} Epochs... Validation RMSLE: {:.6f} +- {:.6f}'.format(
        best_round, search_value, std_eval[best_round]
    ))

    return -search_value



In [25]:
bayes_opt = BayesianOptimization(search_node, BAYESIAN_OPTIMIZATION_BOUNDARIES)
bayes_opt.explore(BAYESIAN_OPTIMIZATION_INITIAL_SEARCH_POINTS)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    bayes_opt.maximize(**BAYESIAN_OPTIMIZATION_MAXIMIZE_PARAMS)

AttributeError: 'BayesianOptimization' object has no attribute 'explore'

In [1]:
from sklearn.model_selection import cross_val_score
import numpy as np

input_dataset = Path("../data/quantile_df.csv")
df = pd.read_csv(input_dataset)
df = df.drop(["x","y"],axis=1)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(df.mean())

y = df['target'] 
X = df.drop("target",axis=1)

def get_estimator_score(learning_rate):    
    estimator = XGBRegressor(objective='reg:squarederror', learning_rate=learning_rate)
    scores = cross_val_score(estimator, X, y, scoring='r2', cv=3)
    return scores.mean()

optimizer = BayesianOptimization(
    f=get_estimator_score,
    pbounds={"learning_rate": (0.1, 10)},
    random_state=1,
)
optimizer.maximize(
    init_points=2,
    n_iter=10
)
print("Final result:", optimizer.max)

## Attempt 2

In [74]:
import numpy as np
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from pathlib import Path
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

pbounds = {
    'learning_rate': (1, 10.0),
    'n_estimators': (20, 200),
    'max_depth': (5,30),
    'gamma': (0, 5),
    'subsample': (0.5,0.9),
    "colsample_bytree": (0.1,1.0)
}

# scores = cross_val_score(XGBRegressor(objective='reg:squarederror',tree_method='gpu_hist', gpu_id=0, n_jobs=-1), X.drop(features_to_drop,axis=1), y, scoring='r2', n_jobs=-1)
def xgboost_hyper_param(learning_rate, n_estimators, max_depth,  gamma, subsample, colsample_bytree):
    max_depth = int(max_depth)
    
    n_estimators = int(n_estimators)
    
    model = XGBRegressor(objective='reg:squarederror',tree_method='gpu_hist', gpu_id=1, n_jobs=-1, min_child_weight=1, scale_pos_weight=1, max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators, gamma=gamma,subsample=subsample, colsample_bytree=colsample_bytree)
    
    return np.mean(cross_val_score(model, X, y, cv=3,scoring='r2', n_jobs=-1))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

In [75]:
input_files = [Path("../data/original_df.csv"),Path("../data/scaled_df.csv"),Path("../data/quantile_df.csv")]   

input_file = input_files[2]
print("input_file: ",input_file)
df = pd.read_csv(input_file)
df = df.drop(["x","y"],axis=1)

y = df['target'].astype('float32') 
X = df.drop("target",axis=1).astype('float32')

optimizer.maximize(
    init_points=100,
    n_iter=50,
)

input_file:  ../data/quantile_df.csv
|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        |  0.1024   |  0.4753   |  3.602    |  1.001    |  12.56    |  46.42    |  0.5369   |
|  2        |  nan      |  0.2676   |  1.728    |  4.571    |  18.47    |  95.46    |  0.7741   |
|  3        | -0.1071   |  0.284    |  4.391    |  1.246    |  21.76    |  95.11    |  0.7235   |
|  4        |  nan      |  0.2263   |  0.9905   |  8.207    |  29.21    |  76.42    |  0.7769   |
|  5        | -0.5583   |  0.8888   |  4.473    |  1.765    |  5.976    |  50.57    |  0.8513   |
|  6        |  nan      |  0.1885   |  2.106    |  9.621    |  18.33    |  144.5    |  0.6262   |
|  7        |  0.09307  |  0.7179   |  4.173    |  1.165    |  23.75    |  198.0    |  0.7993   |
|  8        | -4.05     |  0.3524   |  3.946    |  1.929    |  16.2     |  183.5 

/home/dev/Desktop/Work/uncoverml/venv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


|  45       |  nan      |  0.878    |  1.613    |  7.037    |  16.27    |  88.78    |  0.6643   |
|  46       |  nan      |  0.4613   |  1.587    |  6.597    |  15.76    |  195.3    |  0.7711   |
|  47       |  nan      |  0.2787   |  2.134    |  4.09     |  24.94    |  178.4    |  0.8615   |
|  48       |  nan      |  0.6964   |  1.351    |  3.271    |  26.37    |  115.0    |  0.8209   |
|  49       |  nan      |  0.6152   |  3.666    |  5.671    |  24.27    |  122.4    |  0.6863   |
|  50       |  nan      |  0.4084   |  0.341    |  4.401    |  6.991    |  196.9    |  0.5726   |
|  51       |  nan      |  0.8307   |  4.375    |  7.196    |  19.24    |  48.97    |  0.6868   |
|  52       |  nan      |  0.4107   |  1.125    |  6.333    |  12.81    |  184.9    |  0.8639   |
|  53       |  nan      |  0.3314   |  0.5545   |  2.737    |  17.49    |  151.1    |  0.5833   |
|  54       | -inf      |  0.3232   |  4.258    |  4.743    |  20.42    |  62.06    |  0.5408   |
|  55       | -8.941

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [58]:
optimizer

In [59]:
 optimizer.max

{'target': -0.31142494537734106,
 'params': {'colsample_bytree': 0.4228517846555483,
  'gamma': 3.6016224672107904,
  'learning_rate': 1.001029373356104,
  'max_depth': 12.558314315795993,
  'n_estimators': 21.46755890817113,
  'subsample': 0.5369354379075191}}

In [76]:
 optimizer.max

{'target': nan,
 'params': {'colsample_bytree': 0.2676341902399038,
  'gamma': 1.7278036352152388,
  'learning_rate': 4.570907268076029,
  'max_depth': 18.470418350083925,
  'n_estimators': 95.45501259259306,
  'subsample': 0.7740878001587038}}